In [2]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install tqdm

In [63]:
# 요약보기
pd.set_option('display.max_rows', 100)

In [100]:
#전체행 보기 
pd.set_option('display.max_rows', None)

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re

# 저평가 중소형주 발굴을 통한 투자 전략 수립

## 1. 중소형주 후보군 선정
### 1) 선정기준
- 전체 상장 종목(코스피 + 코스닥) 중 시가총액 하위 30% 추출
- 중소형주 중 최근 1년 수익률 상위 100개 추출

### 2) 시가 총액 하위 30% 추출
#### - 네이버 증권에서 시가 총액 기준으로 크롤링

In [3]:
base_url = "https://finance.naver.com/sise/sise_market_sum.naver"
field_url = "https://finance.naver.com/sise/field_submit.naver"
socks = [0,1]


fields = {'fieldIds': []}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://finance.naver.com/sise/sise_market_sum.naver'
}

session = requests.Session()
session.post(field_url, data=fields, headers=headers)

all_data = []
columns = None
for sosok in tqdm(socks):
  page = 1
  while True:
      response = session.get(f"{base_url}?sosok={sosok}&page={page}", headers=headers)
      response.encoding = 'euc-kr'

      soup = BeautifulSoup(response.text, 'html.parser')
      table = soup.find('table', {'class': 'type_2'})
      
      if page == 1:  # 첫 페이지에서만 컬럼명 가져오기
        columns = []
        for th in table.select('thead th'):
            column = th.text.strip()
            if column:
                columns.append(column)
        columns.append('시장구분')
      
      # 데이터 추출
      page_data = []
      for tr in table.select('tbody tr'):
          row = []
          for td in tr.select('td'):
              value = td.text.strip()
              row.append(value)
          if row and len(row) > 1:  # 빈 행과 구분선 제외
              market_type = "kospi" if sosok == 0 else "kosdaq"
              row.append(market_type)
              page_data.append(row)

      
      # 페이지에 데이터가 없으면 종료
      if not page_data:
          break
          
      all_data.extend(page_data)
      page += 1

  # 전체 데이터를 DataFrame으로 변환
df = pd.DataFrame(all_data, columns=columns)
df['시가총액'] = df['시가총액'].apply(lambda x: int(x.replace(',', '')))
df['거래량'] = df['거래량'].apply(lambda x: int(x.replace(',', '')) if x else 0)
kospi_data = df.loc[df['시장구분'] == 'kospi']
kosdaq_data = df.loc[df['시장구분'] == 'kosdaq']

print(f"kospi_data: {len(kospi_data)}")
print(f"kosdaq_data: {len(kosdaq_data)}")

# 선택한 열만 필터링하고 시가총액 기준으로 정렬
selected_data = df.filter(items=['종목명', '시가총액', '거래량', '시장구분'])
sorted_data = selected_data.sort_values(by='시가총액', ascending=True)

# # CSV 파일로 저장
df.to_csv('stocks_market_cap.csv', index=False, encoding='utf-8-sig')

100%|██████████| 2/2 [00:05<00:00,  2.91s/it]

kospi_data: 2336
kosdaq_data: 1798


In [3]:
df = pd.read_csv('stocks_market_cap.csv', encoding='utf-8-sig')

#### - ETF, ETN, 지수추종, 우선주 등 종목 필터링

In [4]:
# ETF 관련 키워드
filter_ETF = [
    "TIGER", "KODEX", "HANARO", "ARIRANG", 
    "KBSTAR", "KOSEF", "SOL", "BNK", 
    "PLUS", "WON", "FOCUS", "KIWOOM", "ACE",
    "KoAct", "액티브", "S&P", "TREX", "ITF", "나스닥",
    "밸류업", "(H)", "마이티", "코스피"
]

# 인버스/레버리지 관련 키워드
filter_inverse_leverage = [
    "인버스", "레버리지", "선물", "ETN", "RISE"
]

# 우선주/전환사채 관련 키워드
filter_preferred = [
    "우선주", "우B", "전환사채"
]

# 기타 제외할 키워드
filter_etc = [
    "스팩", "리츠", "배당"
]

# 전체 필터링 키워드 통합
filter_keyword = list(set(
    filter_ETF + 
    filter_inverse_leverage + 
    filter_preferred + 
    filter_etc
))


keyword_filtered = sorted_data[~sorted_data['종목명'].apply(lambda x: 
    any(keyword in x for keyword in filter_keyword) or 
    bool(re.search(r'우$', x)) or
    bool(re.search(r'^HK', x))

)]
# 상위 30%의 종목만 필터링
keyword_filtered = keyword_filtered.iloc[0:int(len(sorted_data)*0.3)]

#### - 거래량 필터링

In [5]:
# 거래량 기본 통계 확인
filtered_data = keyword_filtered[keyword_filtered['거래량'].apply(lambda x : x != 0)]

volume_stats = {
    '최대 거래량': filtered_data['거래량'].max(),
    '최소 거래량': filtered_data['거래량'].min(),
    '평균 거래량': filtered_data['거래량'].mean(),
    '중간값': filtered_data['거래량'].median(),
    '표준편차': filtered_data['거래량'].std()
}

# 결과 출력
for key, value in volume_stats.items():
    print(f"{key}: {value:,.0f}")

# 거래량 분포 확인 (사분위수)
print("\n사분위수 분포:")
print(filtered_data['거래량'].describe())

# 중앙값 이상이면서 상위 95%에 해당하는 종목 필터링
volume_filtered = filtered_data[
    (filtered_data['거래량'] >= filtered_data['거래량'].quantile(0.5)) &
    (filtered_data['거래량'] <= filtered_data['거래량'].quantile(0.95)) 
]
volume_filtered

최대 거래량: 229,656,889
최소 거래량: 77
평균 거래량: 752,033
중간값: 49,288
표준편차: 7,949,013

사분위수 분포:
count    1.168000e+03
mean     7.520326e+05
std      7.949013e+06
min      7.700000e+01
25%      1.967200e+04
50%      4.928850e+04
75%      1.500762e+05
max      2.296569e+08
Name: 거래량, dtype: float64


,종목명,시가총액,거래량,시장구분
4063,지엔코,127,167374,kosdaq
4057,빌리언스,131,194418,kosdaq
4052,인베니아,141,52728,kosdaq
4044,더코디,149,86007,kosdaq
4038,비케이홀딩스,162,87205,kosdaq
...,...,...,...,...
3036,빅텍,1180,146380,kosdaq
3035,한빛레이저,1185,128175,kosdaq
3030,다산네트웍스,1186,140104,kosdaq
3032,한선엔지니어링,1186,91677,kosdaq


### 3) 중소형주 중 최근 1년 수익률 상위 100개 추출

In [14]:
from datetime import datetime, timedelta

# 어제 날짜 구하기
yesterday = datetime.now() - timedelta(days=1)

# YYYYMMDD 형식으로 포맷팅
formatted_date = yesterday.strftime("%Y%m%d")

print(formatted_date)  # 예: 20240321 (오늘이 2024년 3월 22일이라고 가정)

one_year_ago = (datetime.now() - timedelta(days=365)).strftime("%Y%m%d")
print(one_year_ago)

20250707
20240708


In [16]:
krx_stock_info

,기준일자,종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,20250707,095570,AJ네트웍스,KOSPI,-,4220,-45,-1.06,4265,4265,4200,113351,478427775,190966642980,45252759
1,20250707,006840,AK홀딩스,KOSPI,-,11910,-250,-2.06,12300,12300,11900,15884,190334730,157778451510,13247561
2,20250707,027410,BGF,KOSPI,-,4170,40,0.97,4140,4245,4115,201001,841238675,399139018470,95716791
3,20250707,282330,BGF리테일,KOSPI,-,124600,3400,2.81,124500,126400,122500,40069,5007420750,2153574687600,17283906
4,20250707,138930,BNK금융지주,KOSPI,-,13750,350,2.61,13610,13920,13400,939708,12907287840,4377773386250,318383519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,20250707,079980,휴비스,KOSPI,-,3330,130,4.06,3215,3380,3205,193249,641700623,114885000000,34500000
958,20250707,005010,휴스틸,KOSPI,-,4690,-35,-0.74,4720,4760,4665,332934,1564665877,263522071750,56188075
959,20250707,000540,흥국화재,KOSPI,-,4170,20,0.48,4150,4200,4080,51436,213315370,267891829650,64242645
960,20250707,000545,흥국화재우,KOSPI,-,7100,-110,-1.53,7190,7190,7100,7971,56661710,5452800000,768000


In [15]:
volume_filtered

,종목명,시가총액,거래량,시장구분
4063,지엔코,127,167374,kosdaq
4057,빌리언스,131,194418,kosdaq
4052,인베니아,141,52728,kosdaq
4044,더코디,149,86007,kosdaq
4038,비케이홀딩스,162,87205,kosdaq
...,...,...,...,...
3036,빅텍,1180,146380,kosdaq
3035,한빛레이저,1185,128175,kosdaq
3030,다산네트웍스,1186,140104,kosdaq
3032,한선엔지니어링,1186,91677,kosdaq


In [9]:
# API endpoint and parameters
kospi_url = "http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd"
kosdaq_url = "http://data-dbg.krx.co.kr/svc/apis/sto/ksq_bydd_trd"
params = {
    "basDd": "20250706"  # Specify the date
}

# Headers with the authentication key
headers = {
    "AUTH_KEY": "18CEC6172A764513B4C60CAA3E3186D3AE2453C3"
}

# Make the GET request
response = requests.get(kospi_url, params=params, headers=headers)

# Check the response status and process the data
if response.status_code == 200:
    data = response.json()
    if 'OutBlock_1' in data:
        krx_stock_info = pd.DataFrame(data['OutBlock_1'])

        # 컬럼명 한글로 변환
        column_map = {
            'BAS_DD': '기준일자',
            'ISU_CD': '종목코드',
            'ISU_NM': '종목명',
            'MKT_NM': '시장구분',
            'SECT_TP_NM': '소속부',
            'TDD_CLSPRC': '종가',
            'CMPPREVDD_PRC': '대비',
            'FLUC_RT': '등락률',
            'TDD_OPNPRC': '시가',
            'TDD_HGPRC': '고가',
            'TDD_LWPRC': '저가',
            'ACC_TRDVOL': '거래량',
            'ACC_TRDVAL': '거래대금',
            'MKTCAP': '시가총액',
            'LIST_SHRS': '상장주식수'
        }

        krx_stock_info = krx_stock_info.rename(columns=column_map)

        numeric_columns = ['종가', '대비', '등락률', '시가', '고가', '저가', '거래량', '거래대금', '시가총액', '상장주식수']
        
        for col in numeric_columns:
            if col in krx_stock_info.columns:
                krx_stock_info[col] = pd.to_numeric(krx_stock_info[col].str.replace(',', ''), errors='coerce')

    print("Data retrieved successfully:")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")
    print("Response:", response.text)

Data retrieved successfully:


In [10]:
response.text

'{"OutBlock_1":[]}'

In [119]:
krx_stock_info[krx_stock_info['종목명'].str.contains('부방', na=False)]

,기준일자,종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
425,20200414,014470,부방,KOSDAQ,우량기업부,2105,105,5.25,1995,2180,1990,656086,1378464940,126410007300,60052260


In [98]:
df["등락률"].iloc[0]

np.float64(1.54)

In [78]:
import requests
import pandas as pd
from datetime import datetime

def get_krx_data(base_date):
    url = "http://data-dbg.krx.co.kr/svc/apis/sto/ksq_bydd_trd"
    headers = {
        "AUTH_KEY": "18CEC6172A764513B4C60CAA3E3186D3AE2453C3"
    }
    params = {
        "basDd": base_date
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            
            # OutBlock_1에서 데이터 추출
            if 'OutBlock_1' in data:
                df = pd.DataFrame(data['OutBlock_1'])
                
                # 컬럼명 한글로 변환
                column_map = {
                    'BAS_DD': '기준일자',
                    'ISU_CD': '종목코드',
                    'ISU_NM': '종목명',
                    'MKT_NM': '시장구분',
                    'SECT_TP_NM': '소속부',
                    'TDD_CLSPRC': '종가',
                    'CMPPREVDD_PRC': '대비',
                    'FLUC_RT': '등락률',
                    'TDD_OPNPRC': '시가',
                    'TDD_HGPRC': '고가',
                    'TDD_LWPRC': '저가',
                    'ACC_TRDVOL': '거래량',
                    'ACC_TRDVAL': '거래대금',
                    'MKTCAP': '시가총액',
                    'LIST_SHRS': '상장주식수'
                }
                df = df.rename(columns=column_map)
                
                # 숫자형 데이터 변환
                numeric_columns = ['종가', '대비', '등락률', '시가', '고가', '저가', 
                                 '거래량', '거래대금', '시가총액', '상장주식수']
                for col in numeric_columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                
                print(f"데이터 로드 완료: {len(df)}개 종목")
                return df
            else:
                print("데이터가 OutBlock_1에 없습니다.")
                return None
            
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return None

# 테스트
today = datetime.now().strftime("%Y%m%d")
df = get_krx_data(today)

if df is not None:
    # 데이터 저장
    df.to_csv('krx_data.csv', index=False, encoding='utf-8-sig')
    
    print("\n데이터 미리보기:")
    print(df.head())
    print("\n데이터 정보:")
    print(df.info())

오류 발생: '종가'


In [2]:
KRX_API_KEY="C197A4D3002F4790A248643FE049703008D57F34"
DART_API_KEY="4758096e577a50f35eaec98b171e463367c7c23f"

In [ ]:
merged_df['시총증감률'].describe()

count    332.000000
mean      -0.095683
std        0.280571
min       -0.791463
25%       -0.271814
50%       -0.114901
75%        0.050438
max        1.291974
Name: 시총증감률, dtype: float64

In [4]:
import requests
import json

# DART API 엔드포인트 설정
url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# API 파라미터 설정
params = {
    'crtfc_key': DART_API_KEY,  # 위에서 정의한 API 키 사용
    'corp_code': '00126380',
    'bsns_year': '2018',
    'reprt_code': '11011',
    'fs_div': 'OFS'
}

# API 요청
response = requests.get(url, params=params)

# JSON 응답 파싱
if response.status_code == 200:
    data = response.json()
    # 데이터 확인
    print("Status:", data['status'])
    print("Message:", data['message'])
    
    # 재무제표 데이터 출력
    for item in data['list']:
        print(f"\nAccount Name: {item['account_nm']}")
        print(f"Amount: {item['thstrm_amount']} {item['currency']}")
else:
    print("Error:", response.status_code)


Status: 000
Message: 정상

Account Name: 유동자산
Amount: 80039455000000 KRW

Account Name: 현금및현금성자산
Amount: 2607957000000 KRW

Account Name: 단기금융상품
Amount: 34113871000000 KRW

Account Name: 매출채권
Amount: 24933267000000 KRW

Account Name: 미수금
Amount: 1515079000000 KRW

Account Name: 선급금
Amount: 807262000000 KRW

Account Name: 선급비용
Amount: 2230628000000 KRW

Account Name: 재고자산
Amount: 12440951000000 KRW

Account Name: 기타유동자산
Amount: 1390440000000 KRW

Account Name: 매각예정분류자산
Amount:  KRW

Account Name: 비유동자산
Amount: 138981902000000 KRW

Account Name: 장기매도가능금융자산
Amount:  KRW

Account Name: 기타포괄손익-공정가치금융자산
Amount: 1098565000000 KRW

Account Name: 당기손익-공정가치금융자산
Amount: 7413000000 KRW

Account Name: 종속기업, 관계기업 및 공동기업 투자
Amount: 55959745000000 KRW

Account Name: 유형자산
Amount: 70602493000000 KRW

Account Name: 무형자산
Amount: 2901476000000 KRW

Account Name: 장기선급비용
Amount: 4108410000000 KRW

Account Name: 순확정급여자산
Amount: 562356000000 KRW

Account Name: 이연법인세자산
Amount: 654456000000 KRW

Account Name: 기타비유

In [7]:
import pandas as pd

# 데이터를 리스트로 변환
data_list = []
for item in data['list']:
    data_list.append({
        '계정명': item['account_nm'],
        '당기금액': item['thstrm_amount'],
        '전기금액': item['frmtrm_amount'],
        '전전기금액': item['bfefrmtrm_amount'],
        '통화': item['currency']
    })

# DataFrame 생성
df = pd.DataFrame(data_list)

# 금액 컬럼들을 숫자로 변환 (빈 문자열은 0으로 처리)
df['당기금액'] = pd.to_numeric(df['당기금액'].replace('', '0'))
df['전기금액'] = pd.to_numeric(df['전기금액'].replace('', '0'))
df['전전기금액'] = pd.to_numeric(df['전전기금액'].replace('', '0'))

# 금액을 보기 좋게 포맷팅 (단위: 백만원)
df['당기금액'] = df['당기금액'] / 1_000_000
df['전기금액'] = df['전기금액'] / 1_000_000
df['전전기금액'] = df['전전기금액'] / 1_000_000

# 결과 출력 (모든 행 표시)
pd.set_option('display.max_rows', None)
print("재무제표 데이터 (단위: 백만원)")
df


재무제표 데이터 (단위: 백만원)


,계정명,당기금액,전기금액,전전기금액,통화
0,유동자산,8.003946e+07,7.015519e+07,6.998113e+07,KRW
1,현금및현금성자산,2.607957e+06,2.763768e+06,3.778371e+06,KRW
2,단기금융상품,3.411387e+07,2.551006e+07,3.017066e+07,KRW
3,매출채권,2.493327e+07,2.788178e+07,2.351401e+07,KRW
4,미수금,1.515079e+06,2.201402e+06,2.319782e+06,KRW
5,선급금,8.072620e+05,1.097598e+06,8.143000e+05,KRW
6,선급비용,2.230628e+06,2.281179e+06,2.375520e+06,KRW
7,재고자산,1.244095e+07,7.837144e+06,5.981634e+06,KRW
8,기타유동자산,1.390440e+06,5.822570e+05,7.431630e+05,KRW
9,매각예정분류자산,0.000000e+00,0.000000e+00,2.836900e+05,KRW


In [8]:
import pandas as pd

# 데이터를 리스트로 변환
data_list = []
for item in data['list']:
    # 재무상태표 항목만 선택 (sj_div가 'BS'인 항목)
    if item['sj_div'] == 'BS':
        data_list.append({
            '계정명': item['account_nm'],
            '당기금액': item['thstrm_amount'],
            '전기금액': item['frmtrm_amount'],
            '전전기금액': item['bfefrmtrm_amount'],
            '통화': item['currency']
        })

# DataFrame 생성
df = pd.DataFrame(data_list)

# 금액 컬럼들을 숫자로 변환 (빈 문자열은 0으로 처리)
df['당기금액'] = pd.to_numeric(df['당기금액'].replace('', '0'))
df['전기금액'] = pd.to_numeric(df['전기금액'].replace('', '0'))
df['전전기금액'] = pd.to_numeric(df['전전기금액'].replace('', '0'))

# 금액을 보기 좋게 포맷팅 (단위: 백만원)
df['당기금액'] = df['당기금액'] / 1_000_000
df['전기금액'] = df['전기금액'] / 1_000_000
df['전전기금액'] = df['전전기금액'] / 1_000_000

# 결과 출력 (모든 행 표시)
pd.set_option('display.max_rows', None)
print("\n재무상태표 데이터 (단위: 백만원)")
display(df)

# 손익계산서 데이터 추출
income_list = []
for item in data['list']:
    # 손익계산서 항목만 선택 (sj_div가 'IS'인 항목)
    if item['sj_div'] == 'IS':
        income_list.append({
            '계정명': item['account_nm'],
            '당기금액': item['thstrm_amount'],
            '전기금액': item['frmtrm_amount'],
            '전전기금액': item['bfefrmtrm_amount'],
            '통화': item['currency']
        })

# DataFrame 생성
df_income = pd.DataFrame(income_list)

# 금액 컬럼들을 숫자로 변환 (빈 문자열은 0으로 처리)
df_income['당기금액'] = pd.to_numeric(df_income['당기금액'].replace('', '0'))
df_income['전기금액'] = pd.to_numeric(df_income['전기금액'].replace('', '0'))
df_income['전전기금액'] = pd.to_numeric(df_income['전전기금액'].replace('', '0'))

# 금액을 보기 좋게 포맷팅 (단위: 백만원)
df_income['당기금액'] = df_income['당기금액'] / 1_000_000
df_income['전기금액'] = df_income['전기금액'] / 1_000_000
df_income['전전기금액'] = df_income['전전기금액'] / 1_000_000

print("\n손익계산서 데이터 (단위: 백만원)")
display(df_income)



재무상태표 데이터 (단위: 백만원)


,계정명,당기금액,전기금액,전전기금액,통화
0,유동자산,80039455.0,70155189.0,69981128.0,KRW
1,현금및현금성자산,2607957.0,2763768.0,3778371.0,KRW
2,단기금융상품,34113871.0,25510064.0,30170656.0,KRW
3,매출채권,24933267.0,27881777.0,23514012.0,KRW
4,미수금,1515079.0,2201402.0,2319782.0,KRW
5,선급금,807262.0,1097598.0,814300.0,KRW
6,선급비용,2230628.0,2281179.0,2375520.0,KRW
7,재고자산,12440951.0,7837144.0,5981634.0,KRW
8,기타유동자산,1390440.0,582257.0,743163.0,KRW
9,매각예정분류자산,0.0,0.0,283690.0,KRW



손익계산서 데이터 (단위: 백만원)


,계정명,당기금액,전기금액,전전기금액,통화
0,수익(매출액),1.703819e+08,1.619150e+08,1.339472e+08,KRW
1,매출원가,1.016665e+08,1.013997e+08,9.729064e+07,KRW
2,매출총이익,6.871536e+07,6.051535e+07,3.665656e+07,KRW
3,판매비와관리비,2.501591e+07,2.565826e+07,2.300912e+07,KRW
4,영업이익(손실),4.369945e+07,3.485709e+07,1.364744e+07,KRW
5,기타수익,9.721450e+05,2.767967e+06,2.185600e+06,KRW
6,기타비용,5.045620e+05,1.065014e+06,1.289594e+06,KRW
7,금융수익,3.737494e+06,4.075602e+06,5.803751e+06,KRW
8,금융비용,3.505673e+06,4.102094e+06,5.622119e+06,KRW
9,법인세비용차감전순이익(손실),4.439886e+07,3.653355e+07,1.472507e+07,KRW


In [9]:
def get_value(df, account_name):
    """특정 계정의 당기 금액을 반환하는 함수"""
    try:
        return df[df['계정명'] == account_name]['당기금액'].values[0]
    except:
        return 0

# 재무비율 계산
당기순이익 = get_value(df_income, '당기순이익(손실)')
자본총계 = get_value(df, '자본총계')
자산총계 = get_value(df, '자산총계')
부채총계 = get_value(df, '부채총계')

# ROE (자기자본이익률) = 당기순이익 / 자기자본 * 100
ROE = (당기순이익 / 자본총계) * 100

# ROA (총자산이익률) = 당기순이익 / 총자산 * 100
ROA = (당기순이익 / 자산총계) * 100

# 부채비율 = 부채총계 / 자본총계 * 100
부채비율 = (부채총계 / 자본총계) * 100

# PER과 PBR은 주가 정보가 필요하므로 KRX API나 다른 소스에서 가져와야 합니다
# 여기서는 계산 방법만 주석으로 표시
# PER = 주가 / 주당순이익(EPS)
# PBR = 주가 / 주당순자산(BPS)

print("\n=== 재무비율 분석 ===")
print(f"ROE (자기자본이익률): {ROE:.2f}%")
print(f"ROA (총자산이익률): {ROA:.2f}%")
print(f"부채비율: {부채비율:.2f}%")

# 데이터프레임으로도 만들어보기
import pandas as pd

ratios = pd.DataFrame({
    '비율': ['ROE (자기자본이익률)', 'ROA (총자산이익률)', '부채비율'],
    '값': [ROE, ROA, 부채비율],
    '단위': ['%', '%', '%']
})

display(ratios)



=== 재무비율 분석 ===
ROE (자기자본이익률): 18.97%
ROA (총자산이익률): 14.98%
부채비율: 26.61%


,비율,값,단위
0,ROE (자기자본이익률),18.969584,%
1,ROA (총자산이익률),14.982615,%
2,부채비율,26.610631,%


In [11]:
# KRX API를 통해 주가 정보 가져오기
import requests

# KRX API 엔드포인트 (예시)
krx_url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

# 삼성전자 종목코드
stock_code = "005930"  # 삼성전자

# API 파라미터
params = {
    'bld': 'dbms/MDC/STAT/standard/MDCSTAT01501',
    'locale': 'ko_KR',
    'tboxisuCd_finder_stkisu0_0': stock_code,
    'isuCd': 'KR7' + stock_code + '00006',
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false'
}

try:
    response = requests.get(krx_url, params=params)
    if response.status_code == 200:
        stock_data = response.json()
        # 주가 데이터 처리
        # 실제 응답 구조에 따라 적절히 수정 필요
        current_price = stock_data.get('주가', 0)  # 예시
        
        # 주당순이익(EPS) 계산
        발행주식수 = get_value(df, '발행주식수')  # 실제 데이터에 있는지 확인 필요
        EPS = 당기순이익 / 발행주식수 if 발행주식수 != 0 else 0
        
        # 주당순자산(BPS) 계산
        BPS = 자본총계 / 발행주식수 if 발행주식수 != 0 else 0
        
        # PER, PBR 계산
        PER = current_price / EPS if EPS != 0 else 0
        PBR = current_price / BPS if BPS != 0 else 0
        
        # 기존 비율 데이터프레임에 추가
        new_ratios = pd.DataFrame({
            '비율': ['PER (주가수익비율)', 'PBR (주가순자산비율)'],
            '값': [PER, PBR],
            '단위': ['배', '배']
        })
        
        ratios = pd.concat([ratios, new_ratios], ignore_index=True)
        display(ratios)
        
except Exception as e:
    print(f"주가 정보를 가져오는데 실패했습니다: {e}")
    print("PER과 PBR은 주가 정보가 필요하므로 계산할 수 없습니다.")
